# Training Script for Probe Models

In [1]:
import evaluate

from data_management import fetch_dataset_from_hf
from training import probing_trainer
from model import ClassifierWithTTA
from transformers import Trainer, TrainingArguments
import numpy as np

import wandb

W0908 16:18:10.913000 14115 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


import error: No module named 'triton'


In [ ]:
train, val = fetch_dataset_from_hf()

## Linear Probe

In [ ]:
model = ClassifierWithTTA(classifier_hidden_layers=0)
model.freeze_embedding()

In [ ]:
trainer = probing_trainer(model, train, val, num_train_epochs=5)

In [ ]:
trainer.train()
wandb.finish()

## ViT probe - 2 layer

In [ ]:
model = ClassifierWithTTA(classifier_hidden_layers=2, classifier_kwargs={
                              'hidden_dropout_prob': 0.3,
                              'attention_probs_dropout_prob': 0.3
                            }
                         )
model.freeze_embedding()

In [ ]:
trainer = probing_trainer(model, train, val, num_train_epochs=5)

In [ ]:
trainer.train()
wandb.finish()

## ViT probe - 4 layer

In [ ]:
model = ClassifierWithTTA(classifier_hidden_layers=4, classifier_kwargs={
                              'hidden_dropout_prob': 0.3,
                              'attention_probs_dropout_prob': 0.3
                            }
                         )
model.freeze_embedding()

In [ ]:
trainer = probing_trainer(model, train, val, num_train_epochs=5)

In [ ]:
trainer.train()
wandb.finish()

# Train script for full models

In [1]:
import evaluate

from data_management import fetch_dataset_from_hf
from training import full_trainer_classification
from model import ClassifierWithTTA
from transformers import Trainer, TrainingArguments
import numpy as np

import wandb

W0913 13:10:57.999000 1153 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


import error: No module named 'triton'


In [2]:
train, val = fetch_dataset_from_hf()

In [3]:
from model import ClassifierWithTTA
from safetensors import safe_open

def load_model_with_dropout(classifier_hidden_layers,
               embedding_kwargs={
                              'hidden_dropout_prob': 0.3,
                              'attention_probs_dropout_prob': 0.3
                            },
               classifier_kwargs={
                              'hidden_dropout_prob': 0.3,
                              'attention_probs_dropout_prob': 0.3
                            }):
    folder = {
                0: "linear_probe",
                2: "vit2_probe",
                4: "vit4_probe"
             }.get(classifier_hidden_layers)
    
    tensors = {}
    with safe_open(f"{folder}/checkpoint-62500/model.safetensors", framework="pt", device="cpu") as f:
        for key in f.keys():
            tensors[key] = f.get_tensor(key)

    model = ClassifierWithTTA(classifier_hidden_layers=classifier_hidden_layers,
                              classifier_kwargs=classifier_kwargs,
                              embedding_kwargs=embedding_kwargs
                             )
    model.load_state_dict(tensors)

    return model

## Linear decoder

In [4]:
linear_model = load_model_with_dropout(0)

In [5]:
trainer = full_trainer_classification(linear_model, train, val,
                                      num_train_epochs=3,
                                      output_dir='linear_full',
                                      run_name='linear_full'
                                     )

In [ ]:
trainer.train()
wandb.finish()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: lrast. Use `wandb login --relogin` to force relogin


/Users/luke/.local/defaultPythonEnv/lib/python3.12/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
W0913 13:12:17.547000 1431 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0913 13:12:17.550000 1429 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0913 13:12:17.550000 1426 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0913 13:12:17.551000 1430 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.
W0913 13:12:17.551000 1433 torch/distributed/elastic/multiprocessing/redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacO

Epoch,Training Loss,Validation Loss


## ViT2 decoder

In [ ]:
vit2_model = load_model_with_dropout(2)

In [ ]:
trainer = full_trainer_classification(vit2_model, train, val,
                                      num_train_epochs=3,
                                      output_dir='vit2_full',
                                      run_name='vit2_full'
                                     )

In [ ]:
trainer.train()
wandb.finish()

## ViT4 decoder

In [ ]:
vit4_model = load_model_with_dropout(4)

In [ ]:
trainer = full_trainer_classification(vit4_model, train, val,
                                      num_train_epochs=3,
                                      output_dir='vit4_full',
                                      run_name='vit4_full'
                                     )

In [ ]:
trainer.train()
wandb.finish()